# 4 STEP TO CREATE A CSV OF TOKYO LAND PRICE LIST

## 1.   Import the Libraries
## 2.   Get CSV file of Tokyo Land Price List(in Japanese)
## 3.   Get Latitude and Longitude Value for Each Land
## 4.   Save DataFrame as a CSV File



### 1. Import the Libraries

In [1]:
# for data processing
import pandas as pd

# for send request
import requests

# for displaying a progress bar
from tqdm.notebook import tqdm

# for user input for your Google Api Key
from getpass import getpass

# for downloading csv file(if you use Google Colab)
from google.colab import files

### 2. Get CSV file of Tokyo Land Price List and Save it to DataFrame

In [2]:
# get csv file from Tokyo metropolitan page
url = 'https://www.zaimu.metro.tokyo.lg.jp/kijunchi/R2kouji/R2kouji_chiten_data.csv'
price_df_jp = pd.read_csv(url, encoding='cp932', header=1)

# change the name of columns
price_df_jp = price_df_jp[['区市町村名', '地番', '当年価格（円）']]
price_df_jp.columns = ['ward_name', 'address_detail', 'land_price']

# drop nan and change datatype of two columns to int
price_df_jp.dropna(axis=0, inplace=True)
price_df_jp['land_price'] = price_df_jp['land_price'].str.replace(',','').astype(int)
price_df_jp.head()

,ward_name,address_detail,land_price
0,千代田区,三番町６番２５,3160000
1,千代田区,紀尾井町３番２７外,2020000
2,千代田区,六番町６番１外,4050000
3,千代田区,富士見１丁目８番６,1550000
4,千代田区,九段北２丁目６番２６,2960000


### 3. Get Latitude and Longitude Value for Each Land

To use Google Geocoding API, you need to have API KEY to be set up.<br>
Check the link below for more information.
<br><br>
<a href="https://developers.google.com/maps/documentation/geocoding/get-api-key" target="_blank">Google Geocoding API Documentation</a>
<hr>



In [3]:
# input your Google Api key
API_KEY = getpass('INPUT YOUR GOOGLE API KEY \nAND PRESS ENTER: ')

INPUT YOUR GOOGLE API KEY 
AND PRESS ENTER: ··········


In [4]:
# geocoding from address to location data
land_values_list = []
row_count = price_df_jp.shape[0]

for i in tqdm(range(row_count)):
  row = price_df_jp.iloc[i]
  address = row[0] + row[1]
  link = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(address, API_KEY)
  r = requests.get(link)
  map_json = r.json()
  if map_json['status'] == 'ZERO_RESULTS':
    lat, lon, area_name, address_detail_en, land_price = None, None, None, None, None
  else:
    for area in map_json['results'][0]['address_components']:
      if 'locality' in area['types']:
        area_name = area['long_name']
    lat = map_json['results'][0]['geometry']['location']['lat']
    lon = map_json['results'][0]['geometry']['location']['lng']
    address_detail_en = map_json['results'][0]['formatted_address']
    land_price = row[2]
  land_values_list.append([area_name, address_detail_en, lat, lon, land_price])

### 4. Save DataFrame as a CSV File

In [5]:
# create a new DataFrame
price_df_en = pd.DataFrame(land_values_list, columns=['area_name', 'address_detail', 'lat', 'lon', 'land_price_per_square_meter'])
price_df_en.head()

,area_name,address_detail,lat,lon,land_price_per_square_meter
0,Chiyoda City,"6-25 Sanbanchō, Chiyoda City, Tōkyō-to 102-007...",35.690086,139.744837,3160000.0
1,Chiyoda City,"3-27 Kioichō, Chiyoda City, Tōkyō-to 102-0094,...",35.682015,139.737615,2020000.0
2,Chiyoda City,"6-1 Rokubanchō, Chiyoda City, Tōkyō-to 102-008...",35.688143,139.732869,4050000.0
3,Chiyoda City,"1-chōme-8-6 Fujimi, Chiyoda City, Tōkyō-to 102...",35.697784,139.745095,1550000.0
4,Chiyoda City,"2 Chome Kudankita, Chiyoda City, Tokyo 102-007...",35.695576,139.746362,2960000.0


In [6]:
# save as a csv file to local storage
price_df_en.to_csv('tokyo_land_price_data.csv')
files.download('tokyo_land_price_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>